# Sentiment Analysis를 통한 강의평 기반 강의 추천

시작 전 주의!

런타임-런타임 유형 변경-GPU로 바꿔주고 시작합니다.

필요한 파일은 2020LectureReview.csv 입니다.

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 39.7MB/s 
     |████████████████████████████████| 901kB 52.3MB/s 


In [ ]:
#주요 패키지들을 불러와줍니다.

import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
#Torch에서 GPU가 사용되는지 확인하는 코드입니다. 어떤 GPU를 쓰는지 확인할 수 있습니다.

import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


#데이터셋

In [ ]:
data = pd.read_csv("2020LectureReview.csv", index_col = 0)
data = data.dropna(axis=0) #null 있는 행 제거
data.shape

(76501, 6)

In [ ]:
# 감성분석 모델링을 위해 별점 1,2점을 받은 리뷰는 0으로, 별점 4,5 점을 받은 리뷰는 1로 대치.
change_value_dict = {100:1, 80:1, 60:60, 40:0, 20:0}
data = data.replace({'ReviewStar':change_value_dict})
data = data.astype({'ReviewStar':'int'}) #float -> int

# 별점 3점을 받은 리뷰는 감성분석 모델링을 위해 제거 
data = data[data.ReviewStar != 60]
data.shape

(64600, 6)

In [ ]:
#train test 분리
train = data.iloc[:50000]
test1 = data.iloc[50000:]
trainfortest2 = train #copy

print(train.shape, test1.shape)

(50000, 6) (14600, 6)


In [ ]:
#[CLS] [SEP] 토큰 포함 128자 이하의 문장들로 학습시켰습니다.
index = train["ReviewText"].str.len() > 126
index_2 = trainfortest2["ReviewText"].str.len() <= 126

train = train.loc[~index]
test2 = trainfortest2.loc[~index_2] #토큰 포함 128자 이상의 문장들로만 구성된 테스트셋

print(train.shape, test2.shape)

(27408, 6) (22592, 6)


# 문장별 전처리

Bert 분류 모델은 각 문장의 앞마다 [CLS]를 붙여 인식합니다. 문장 종료는 [SEP]로 알립니다. 이를 처리해줍니다.

In [ ]:
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.ReviewText]
document_bert[:5]

['[CLS] 시험 잘 멋본 것 같은데 에이제로 떠서 놀람 감사합니다 교수님 [SEP]',
 '[CLS] 에 뭐,,, 그냥 괜찮았는데 자습기간을 안 주고 전범위 시험본다그러니까 당황스러웠네요. 교수님 똑똑하시고 나쁘지 않아요. 다만 3시간 연속인 게 좀 힘들지만 뭐 그건 당연히 고려하시고 신청하시겠죠. [SEP]',
 '[CLS] 교수님 너무 똑똑하시고 피피티 위주로 수업하시기는 하는데 현장경험이 풍부하셔서 그런지 잘 설명해주십니다 저는 정말 좋았어요 [SEP]',
 '[CLS] 조교가 일을 안하는건지 교수가 일을 안하는건지 강의가 맨날 늦게 올라옴 제 시간에 강의를 안 보게 되니까 이 수업에 대한 열정이 초반부터 확 사라짐. 그러면서 저작권 때문에 강의는 일찍 내려야겠다고 함 ㅋㅋ [SEP]',
 '[CLS] 교수님의 수업은 이해하기 쉬웠고 질적연구와 현장연구에 대해 잘 설명해주셔서 수업은 매우 좋았지만, 중간고사 비중이 크고 성적 산출에 있어서 아쉬운 부분이 있습니다. [SEP]']

# 토크나이징
사전학습된 BERT multilingual 모델 내 포함되어있는 토크나이저를 활용하여 토크나이징합니다.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])


['[CLS]', '시', '##험', '잘', '멋', '##본', '것', '같은', '##데', '에', '##이', '##제로', '떠', '##서', '놀', '##람', '감', '##사', '##합', '##니다', '교', '##수', '##님', '[SEP]']


#패딩
token들의 max length보다 크게 max_len 값을 설정합니다. 설정한 max_len만큼 문장 길이에 따른 빈 공간을 0이 채웁니다. 이를 패딩이라 합니다.

In [ ]:
# 패딩 작업
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([   101,   9485,  86834,   9654,   9270,  40419,   8870,  18589,
        28911,   9559,  10739,  53914,   9138,  12424,   9025,  61250,
         8848,  12945,  33188,  48345,   8907,  15891, 108578,    102,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

# 어텐션 마스크
학습 속도를 높이기 위해 실 데이터가 있는 곳과 padding이 있는 곳을 attention에게 알려줍니다.

In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# train-validation set 분리
random state는 일정히 고정합니다.(input과 mask가 섞이지 않기 위함)

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, train['ReviewStar'].values, random_state=42, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=42, 
                                                       test_size=0.1)

#파이토치 텐서로 변환
numpy ndarray로 되어 있는 inputs, labels, masks들을 torch tensor로 변환합니다.

In [ ]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

# 배치 및 데이터로더 설정
현재 쓰고 있는 GPU의 VRAM에 맞도록 배치 사이즈를 설정합니다. 우선 배치사이즈를 크게 넣어보고 VRAM 부족 메세지가 나오면 8의 배수 중 더 작은 것으로 줄여나갑니다.

In [ ]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

#테스트셋 전처리
위에서 trainset 전처리 한것과 동일한 과정을  testset에 반복합니다.

In [ ]:
#테스트 데이터 전처리 과정 : test1셋(128자 구분 없이 구성된 테스트셋)

sentences = test1['ReviewText']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test1['ReviewStar'].values
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
#테스트 데이터 전처리 과정 : test2셋(128자 이상 text로만 구성된 테스트셋)

sentences = test2['ReviewText']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test2['ReviewStar'].values
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader_2 = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

#모델 학습

In [ ]:
#GPU 체크 및 할당
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# cuda error 뜰 때만 쓰는 코드
# import torch, gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
#bert 모델 생성

#transformers의 bertforsequenceclassification 모듈 이용.

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda() #gpu 사용할 때는 주석 풀어주세요

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 학습 스케쥴링
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

#학습

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 아래는 학습 실행 부분입니다. 한 시간정도 소요됩니다.
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of    771.    Elapsed: 0:06:05.

  Average training loss: 0.29
  Training epcoh took: 0:09:28

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:22

======== Epoch 2 / 4 ========
Training...
  Batch   500  of    771.    Elapsed: 0:06:15.

  Average training loss: 0.17
  Training epcoh took: 0:09:38

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:22

======== Epoch 3 / 4 ========
Training...
  Batch   500  of    771.    Elapsed: 0:06:15.

  Average training loss: 0.10
  Training epcoh took: 0:09:37

Running Validation...
  Accuracy: 0.95
  Validation took: 0:00:22

======== Epoch 4 / 4 ========
Training...
  Batch   500  of    771.    Elapsed: 0:06:13.

  Average training loss: 0.07
  Training epcoh took: 0:09:35

Running Validation...
  Accuracy: 0.95
  Validation took: 0:00:22

Training complete!


#테스트셋 평가

In [ ]:
################# test1 세트 평가(128자 구분 없는 테스트 세트) #################

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    457.    Elapsed: 0:00:27.
  Batch   200  of    457.    Elapsed: 0:00:54.
  Batch   300  of    457.    Elapsed: 0:01:19.
  Batch   400  of    457.    Elapsed: 0:01:45.

Accuracy: 0.92
Test took: 0:02:00


In [ ]:
################# test2 세트 평가(128자 이상으로만 구성된 테스트 세트) #################

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader_2):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader_2), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    706.    Elapsed: 0:00:26.
  Batch   200  of    706.    Elapsed: 0:00:52.
  Batch   300  of    706.    Elapsed: 0:01:18.
  Batch   400  of    706.    Elapsed: 0:01:45.
  Batch   500  of    706.    Elapsed: 0:02:11.
  Batch   600  of    706.    Elapsed: 0:02:37.
  Batch   700  of    706.    Elapsed: 0:03:03.

Accuracy: 0.90
Test took: 0:03:05


# 모델 저장

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.externals import joblib
joblib.dump(model, 'model.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['model.pkl']